In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
class MNISTDataset(Dataset):
    def __init__(self, csv_file, has_labels=True):
        data = pd.read_csv(csv_file).values
        
        self.x = torch.tensor(data[:, 1:].reshape(-1, 1, 28, 28), dtype=torch.float32) / 255.0
        self.y = torch.tensor(data[:, 0], dtype=torch.long) 
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),                            
            nn.Conv2d(32, 64, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),                            
            nn.Flatten(),                               
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.model(x)

def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss, correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
    accuracy = correct / len(train_loader.dataset)
    return total_loss / len(train_loader), accuracy

def predict_test(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in test_loader:  
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions

def main():
    # Load datasets
    train_dataset = MNISTDataset('mnist_train.csv', has_labels=True)
    test_dataset = MNISTDataset('mnist_test.csv', has_labels=False)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNN().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(1, 6):
        print(f"Epoch {epoch}")
        train_loss, train_accuracy = train_model(model, train_loader, criterion, optimizer, device)
        print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    predictions = predict_test(model, test_loader, device)

    submission = pd.DataFrame({
        'ImageId': range(1, len(predictions) + 1),
        'Label': predictions
    })
    submission.to_csv('submission.csv', index=False)
    print("Predictions saved to submission.csv")

if __name__ == '__main__':
    main()

Epoch 1
Train Loss: 0.1711, Accuracy: 0.9474
Epoch 2
